In [ ]:
import pandas as pd
import numpy as np

## Section Introduction

This section will focus on **Pivot Tables**. The term was trademarked by Microsoft in the 90s for Microsoft Excel. Pandas pivot tables will draw heavily from this. This section will cover:
* building pivot tables from first principles
* the `pivot()` and `pivot_table()` constructs
* multiindex pivot tables
* applying multiple functions
* pivot cosumization with margins, multiple functions, and more!

## New Data: New York City SAT Scores

In this section we will analyze SAT scores from New York City high schools from the year 2012. For this course, the instructor has made some changes to the original data, including removing schools with missing data, removing certain attributes that are not of interest to us, and reshaping the data to prepare for this section.
* https://andybek.com/pandas-sat

In [ ]:
sat_url = 'https://andybek.com/pandas-sat'

In [ ]:
sat = pd.read_csv(sat_url)

In [ ]:
sat.head()

,School Name,Borough,Student Enrollment,Percent Tested,SAT Section,Score
0,A. Philip Randolph Campus High School,Manhattan,1354,74.00%,Math,459
1,A. Philip Randolph Campus High School,Manhattan,1354,74.00%,Reading,453
2,A. Philip Randolph Campus High School,Manhattan,1354,74.00%,Writing,447
3,Abraham Lincoln High School,Brooklyn,2341,54.90%,Math,441
4,Abraham Lincoln High School,Brooklyn,2341,54.90%,Reading,422


In [ ]:
sat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1125 entries, 0 to 1124
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   School Name         1125 non-null   object
 1   Borough             1125 non-null   object
 2   Student Enrollment  1125 non-null   int64 
 3   Percent Tested      1125 non-null   object
 4   SAT Section         1125 non-null   object
 5   Score               1125 non-null   int64 
dtypes: int64(2), object(4)
memory usage: 52.9+ KB


So we have a 6-column dataframe, two of which are numeric ("Student Enrollment" and "Score"). Note that because of the percentage sign, "Percent Tested" is an `object` datatype instead of numeric.

Let's start by replacing the "Percent Tested" with a floating point value.

In [ ]:
sat['Percent Tested'] = sat['Percent Tested'].replace(regex='%', value = '').astype(float)

In [ ]:
sat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1125 entries, 0 to 1124
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   School Name         1125 non-null   object 
 1   Borough             1125 non-null   object 
 2   Student Enrollment  1125 non-null   int64  
 3   Percent Tested      1125 non-null   float64
 4   SAT Section         1125 non-null   object 
 5   Score               1125 non-null   int64  
dtypes: float64(1), int64(2), object(3)
memory usage: 52.9+ KB


Beautiful. Let's get started.

## Pivoting Data with the `pivot()` Method

Pivoting is all about reshaping data. We take a dataframe or series, and without transforming the content, we present it differently. Changing the structure of a dataset depending on the question we are trying to answer is a very important skill in data analysis.

In [ ]:
sat.head(9)

,School Name,Borough,Student Enrollment,Percent Tested,SAT Section,Score
0,A. Philip Randolph Campus High School,Manhattan,1354,74.0,Math,459
1,A. Philip Randolph Campus High School,Manhattan,1354,74.0,Reading,453
2,A. Philip Randolph Campus High School,Manhattan,1354,74.0,Writing,447
3,Abraham Lincoln High School,Brooklyn,2341,54.9,Math,441
4,Abraham Lincoln High School,Brooklyn,2341,54.9,Reading,422
5,Abraham Lincoln High School,Brooklyn,2341,54.9,Writing,422
6,Academy for Careers in Television and Film,Queens,515,95.0,Math,444
7,Academy for Careers in Television and Film,Queens,515,95.0,Reading,458
8,Academy for Careers in Television and Film,Queens,515,95.0,Writing,444


Consider our *sat* dataset. Each school's scores in Math, Reading, and Writing is represented in a separate row. This is not ideal because it repeats school names, enrollment, and borough several times for each school. We can argue that this is not the most sensible way to present the data.

Let's change the structure of the dataframe to move the "SAT Section" to the column axis, so that we have separate columns for Math, Reading and Writing. The school names will be the index, and the scores will be the sequence of values. This is not the only way to reshape, but it is one option.

In [ ]:
sat.set_index(['School Name', "SAT Section"])

Borough  ...  Score
School Name                                SAT Section             ...       
A. Philip Randolph Campus High School      Math         Manhattan  ...    459
                                           Reading      Manhattan  ...    453
                                           Writing      Manhattan  ...    447
Abraham Lincoln High School                Math          Brooklyn  ...    441
                                           Reading       Brooklyn  ...    422
...                                                           ...  ...    ...
Young Women's Leadership School in Astoria Reading         Queens  ...    464
                                           Writing         Queens  ...    477
Young Women's Leadership School in Queens  Math            Queens  ...    415
                                           Reading         Queens  ...    420
                                           Writing         Queens  ...    433

[1125 rows x 4 columns]

We can now do label-based selection for the scores (we lost the rest of the data, but we don't care too much about that for now).

In [ ]:
sat.set_index(['School Name', "SAT Section"]).loc[:,['Score']]

Score
School Name                                SAT Section       
A. Philip Randolph Campus High School      Math           459
                                           Reading        453
                                           Writing        447
Abraham Lincoln High School                Math           441
                                           Reading        422
...                                                       ...
Young Women's Leadership School in Astoria Reading        464
                                           Writing        477
Young Women's Leadership School in Queens  Math           415
                                           Reading        420
                                           Writing        433

[1125 rows x 1 columns]

Finally, let's kick the "SAT Section" level of the index over to the column axis using `unstack()`. 
* https://pandas.pydata.org/pandas-docs/version/1.3.4/reference/api/pandas.DataFrame.unstack.html

In [ ]:
sat.set_index(['School Name', "SAT Section"]).loc[:,['Score']].unstack(level = 'SAT Section')

Score                
SAT Section                                         Math Reading Writing
School Name                                                             
A. Philip Randolph Campus High School                459     453     447
ACORN Community High School                          379     395     385
Abraham Lincoln High School                          441     422     422
Academy for Careers in Television and Film           444     458     444
Academy for College Preparation and Career Expl...   386     397     393
...                                                  ...     ...     ...
World Journalism Preparatory (College Board)         484     491     487
York Early College Academy                           496     481     473
Young Women's Leadership School                      478     465     472
Young Women's Leadership School in Astoria           483     464     477
Young Women's Leadership School in Queens            415     420     433

[375 rows x 3 columns]

We now have a much nicer presentation of the same data. But we arguably had to go through a lot of steps to get there. We had to change the index, then select the "Score" column, and finally unpivoted one level of the index. 

This kind of reshaping work is so common that Pandas offers a dedicated method for accomplishing this in one single command. Starting with our raw dataframe, we can call the `pivot()` method as follows:
* https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.pivot.html

In [ ]:
sat.pivot('School Name','SAT Section','Score')

SAT Section,Math,Reading,Writing
School Name,,,
A. Philip Randolph Campus High School,459,453,447
ACORN Community High School,379,395,385
Abraham Lincoln High School,441,422,422
Academy for Careers in Television and Film,444,458,444
Academy for College Preparation and Career Exploration (College Board),386,397,393
...,...,...,...
World Journalism Preparatory (College Board),484,491,487
York Early College Academy,496,481,473
Young Women's Leadership School,478,465,472


That's a thing of beauty right there. We've condensed a 3-step process to a single line of code.

Given that this is our first time using `pivot()`, it would behoove us to include the parameter names instead of relying on positional arguments.

In [ ]:
sat.pivot(index = 'School Name', columns = 'SAT Section', values = 'Score')

SAT Section,Math,Reading,Writing
School Name,,,
A. Philip Randolph Campus High School,459,453,447
ACORN Community High School,379,395,385
Abraham Lincoln High School,441,422,422
Academy for Careers in Television and Film,444,458,444
Academy for College Preparation and Career Exploration (College Board),386,397,393
...,...,...,...
World Journalism Preparatory (College Board),484,491,487
York Early College Academy,496,481,473
Young Women's Leadership School,478,465,472


If this looks familiar to Excel users, it should. It's exactly what the pivot tables in Excel do. We want to do more analysis here, so we'll assign this output to a variable.


In [ ]:
pivoted = sat.pivot(index = 'School Name', columns = 'SAT Section', values = 'Score')

In [ ]:
pivoted.head()

SAT Section,Math,Reading,Writing
School Name,,,
A. Philip Randolph Campus High School,459,453,447
ACORN Community High School,379,395,385
Abraham Lincoln High School,441,422,422
Academy for Careers in Television and Film,444,458,444
Academy for College Preparation and Career Exploration (College Board),386,397,393


## Jason's Side Project

### Which borough had the highest average scores on per-SAT-Section basis?

In [ ]:
sat.groupby(["Borough", "SAT Section"])['Score'].mean().sort_values(ascending = False)

Borough        SAT Section
Staten Island  Math           486.200000
               Reading        478.500000
               Writing        474.300000
Queens         Math           462.362319
Manhattan      Math           455.887640
               Reading        444.932584
Queens         Reading        443.260870
               Writing        439.855072
Manhattan      Writing        439.314607
Brooklyn       Math           416.403670
               Reading        410.761468
Bronx          Math           404.357143
Brooklyn       Writing        403.091743
Bronx          Reading        402.520408
               Writing        395.846939
Name: Score, dtype: float64

### Which school had the highest scores?

In [ ]:
sat.set_index(['School Name','SAT Section']).sort_values(by = "Score", ascending = False)

Borough  ...  Score
School Name                                        SAT Section                 ...       
Stuyvesant High School                             Math             Manhattan  ...    754
Bronx High School of Science                       Math                 Bronx  ...    714
Staten Island Technical High School                Math         Staten Island  ...    711
Queens High School for the Sciences at York Col... Math                Queens  ...    701
Stuyvesant High School                             Reading          Manhattan  ...    697
...                                                                       ...  ...    ...
Kingsbridge International High School              Writing              Bronx  ...    310
International High School at Prospect Heights      Reading           Brooklyn  ...    302
                                                   Writing           Brooklyn  ...    300
Pan American International High School at Monroe   Writing              Bronx  ...    292
Multicultural High School                          Writing           Brooklyn  ...    284

[1125 rows x 4 columns]

## Undoing Pivots with `melt()`

What happens if we want to "undo" a pivot?

One approach is to assign different pivot views to different variables, so that if we want to revert back to a previous view, we simply call that variable. However this is considered cheating because we're not really "undoing" anything - we're just going back to a previous dataframe stored in memory. 

So how do we do this legitimately? Let's begin with a pivoted dataframe from last lecture.

In [ ]:
pivoted

SAT Section,Math,Reading,Writing
School Name,,,
A. Philip Randolph Campus High School,459,453,447
ACORN Community High School,379,395,385
Abraham Lincoln High School,441,422,422
Academy for Careers in Television and Film,444,458,444
Academy for College Preparation and Career Exploration (College Board),386,397,393
...,...,...,...
World Journalism Preparatory (College Board),484,491,487
York Early College Academy,496,481,473
Young Women's Leadership School,478,465,472


Suppose that we want to bring the SAT Section column axis back to a single column, like we had at the beginning of this section. An easy way to do this is to call `stack()`, which takes the column axis and rotates it clockwise into the right-most level of the index axis, making it longer and narrower (hence the term "stack". 

In [ ]:
pivoted.stack()

School Name                                 SAT Section
A. Philip Randolph Campus High School       Math           459
                                            Reading        453
                                            Writing        447
ACORN Community High School                 Math           379
                                            Reading        395
                                                          ... 
Young Women's Leadership School in Astoria  Reading        464
                                            Writing        477
Young Women's Leadership School in Queens   Math           415
                                            Reading        420
                                            Writing        433
Length: 1125, dtype: int64

Again, this pushes "SAT Section" into the multiindex.

In [ ]:
pivoted.stack().index

MultiIndex([(     'A. Philip Randolph Campus High School',    'Math'),
            (     'A. Philip Randolph Campus High School', 'Reading'),
            (     'A. Philip Randolph Campus High School', 'Writing'),
            (               'ACORN Community High School',    'Math'),
            (               'ACORN Community High School', 'Reading'),
            (               'ACORN Community High School', 'Writing'),
            (               'Abraham Lincoln High School',    'Math'),
            (               'Abraham Lincoln High School', 'Reading'),
            (               'Abraham Lincoln High School', 'Writing'),
            ('Academy for Careers in Television and Film',    'Math'),
            ...
            (                'York Early College Academy', 'Writing'),
            (           'Young Women's Leadership School',    'Math'),
            (           'Young Women's Leadership School', 'Reading'),
            (           'Young Women's Leadership School', 'W

To revert back to a flat dataframe, we can chain on a `set_index()` call.

In [ ]:
pivoted.stack().reset_index()

,School Name,SAT Section,0
0,A. Philip Randolph Campus High School,Math,459
1,A. Philip Randolph Campus High School,Reading,453
2,A. Philip Randolph Campus High School,Writing,447
3,ACORN Community High School,Math,379
4,ACORN Community High School,Reading,395
...,...,...,...
1120,Young Women's Leadership School in Astoria,Reading,464
1121,Young Women's Leadership School in Astoria,Writing,477
1122,Young Women's Leadership School in Queens,Math,415
1123,Young Women's Leadership School in Queens,Reading,420


So there we have it - a flat dataframe with a standard range index.

Another way to get this done is to use the `melt()` method. The `melt()` method is used to change a dataframe to a structure where the so-called *measured variables* are unpivoted into the row axis. 
* https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.melt.html
* Any variables that are not *measured variables* are considered *identifier variables*. Identifier variables are not forced into the unpivoted structure.
* All measured variable columns are stacked by the `melt()` method
* At the end of the method, you have identifier columns, one variable column, and one value column

In [ ]:
pivoted.head()

SAT Section,Math,Reading,Writing
School Name,,,
A. Philip Randolph Campus High School,459,453,447
ACORN Community High School,379,395,385
Abraham Lincoln High School,441,422,422
Academy for Careers in Television and Film,444,458,444
Academy for College Preparation and Career Exploration (College Board),386,397,393


In [ ]:
pivoted.melt()

,SAT Section,value
0,Math,459
1,Math,379
2,Math,441
3,Math,444
4,Math,386
...,...,...
1120,Writing,487
1121,Writing,473
1122,Writing,472
1123,Writing,477


Notice how the measured variable "SAT Section" has been unpivoted from the column axis into the row axis. In the process however we lost our "School Names" which were in the index.

To ensure "School Names" stick around, we can reset the index before performing the melt.


In [ ]:
pivoted.reset_index().head()

SAT Section,School Name,Math,Reading,Writing
0,A. Philip Randolph Campus High School,459,453,447
1,ACORN Community High School,379,395,385
2,Abraham Lincoln High School,441,422,422
3,Academy for Careers in Television and Film,444,458,444
4,Academy for College Preparation and Career Exp...,386,397,393


In [ ]:
pivoted.reset_index().melt()

,SAT Section,value
0,School Name,A. Philip Randolph Campus High School
1,School Name,ACORN Community High School
2,School Name,Abraham Lincoln High School
3,School Name,Academy for Careers in Television and Film
4,School Name,Academy for College Preparation and Career Exp...
...,...,...
1495,Writing,487
1496,Writing,473
1497,Writing,472
1498,Writing,477


Well that's weird - "School Name" has been considered a measured variable by Pandas, whereas it should be considered an identifier variable. As an identifer variable, it would not be forced into the unpivoted structure. How do we fix this?

In [ ]:
pivoted.reset_index().melt(id_vars = "School Name")

,School Name,SAT Section,value
0,A. Philip Randolph Campus High School,Math,459
1,ACORN Community High School,Math,379
2,Abraham Lincoln High School,Math,441
3,Academy for Careers in Television and Film,Math,444
4,Academy for College Preparation and Career Exp...,Math,386
...,...,...,...
1120,World Journalism Preparatory (College Board),Writing,487
1121,York Early College Academy,Writing,473
1122,Young Women's Leadership School,Writing,472
1123,Young Women's Leadership School in Astoria,Writing,477


That's much better - we have stacked "SAT Section" scores but kept the "School Name" as it was before the unpivot.

A final note is that when we use `melt()`, we always create a sequence of measured variables stacked in one column and their measured values. This measured value column is always called "value". If we want to change this to something more meaningful, we can use the `value_name` parameter within the `melt()` function.

In [ ]:
pivoted.reset_index().melt(id_vars = "School Name", value_name = "Score")

,School Name,SAT Section,Score
0,A. Philip Randolph Campus High School,Math,459
1,ACORN Community High School,Math,379
2,Abraham Lincoln High School,Math,441
3,Academy for Careers in Television and Film,Math,444
4,Academy for College Preparation and Career Exp...,Math,386
...,...,...,...
1120,World Journalism Preparatory (College Board),Writing,487
1121,York Early College Academy,Writing,473
1122,Young Women's Leadership School,Writing,472
1123,Young Women's Leadership School in Astoria,Writing,477


## What About Aggregates?

The reshaping we have done so far has just realigned the index and column axes, resulting in a reordering of values. The values themselves have not been analyzed or used to perform any calculations; all we've done is change how they are displayed. Compare our *sat* and *pivoted* tables.


In [ ]:
pivoted.head(3)

SAT Section,Math,Reading,Writing
School Name,,,
A. Philip Randolph Campus High School,459,453,447
ACORN Community High School,379,395,385
Abraham Lincoln High School,441,422,422


In [ ]:
sat.head(3)

,School Name,Borough,Student Enrollment,Percent Tested,SAT Section,Score
0,A. Philip Randolph Campus High School,Manhattan,1354,74.0,Math,459
1,A. Philip Randolph Campus High School,Manhattan,1354,74.0,Reading,453
2,A. Philip Randolph Campus High School,Manhattan,1354,74.0,Writing,447


Now suppose we are interested in getting some insight on the overall SAT scores by borough (administrative division of New York City).

In [ ]:
sat.Borough.unique()

array(['Manhattan', 'Brooklyn', 'Queens', 'Bronx', 'Staten Island'],
      dtype=object)

Let's answer the following question: What is the average SAT score by borough?

Let's first try to approach this by pivoting our dataframe such that "Borough" is the index, "SAT Section" is columns, and the "Score" is values. Spoiler: this does not work.

In [ ]:
## This results in a ValueError: Index contains duplicate entries, cannot reshape
# sat.pivot(index = "Borough", columns = 'SAT Section', values = 'Score')

When we try that approach we get a ValueError due to the fact that the "Borough" column has repeat entries. In the structure that we've declared, in which the "SAT Section" column is pivoted into the column axis, the index values will not be unique - each borough has multiple schools, and thus borough names would appear multiple times.

Why is this an issue? After all, we know that indexes can have repeat values. For instance, we can easily set "Borough" as the index of our normal dataframe.

In [ ]:
sat.set_index("Borough")

,School Name,Student Enrollment,Percent Tested,SAT Section,Score
Borough,,,,,
Manhattan,A. Philip Randolph Campus High School,1354,74.0,Math,459
Manhattan,A. Philip Randolph Campus High School,1354,74.0,Reading,453
Manhattan,A. Philip Randolph Campus High School,1354,74.0,Writing,447
Brooklyn,Abraham Lincoln High School,2341,54.9,Math,441
Brooklyn,Abraham Lincoln High School,2341,54.9,Reading,422
...,...,...,...,...,...
Queens,Young Women's Leadership School in Astoria,572,100.0,Reading,464
Queens,Young Women's Leadership School in Astoria,572,100.0,Writing,477
Queens,Young Women's Leadership School in Queens,557,90.4,Math,415


So what's the problem? Other than being a bad practice, there's no issue with repeated index values. 

However, when it comes to the Pandas `pivot()` method, we have to be more thoughtful of how this works. In this specific case, we cannot pivot "SAT Section" into the column axis while assigning "borough" to the index because each Borough has multiple sets of "SAT Section" scores due to the fact that each Borough contains multiple schools. Pandas gets confused when you ask it to report the scores on a per-borough basis when each borough has multiple scores due to multiple schools.

On the other hand, we *CAN* pivot "SAT Section" into the column axis and assign "School Name" to the index because each school has only one set of SAT scores. 

In [ ]:
sat.pivot(index = "School Name", columns = 'SAT Section', values = 'Score')

SAT Section,Math,Reading,Writing
School Name,,,
A. Philip Randolph Campus High School,459,453,447
ACORN Community High School,379,395,385
Abraham Lincoln High School,441,422,422
Academy for Careers in Television and Film,444,458,444
Academy for College Preparation and Career Exploration (College Board),386,397,393
...,...,...,...
World Journalism Preparatory (College Board),484,491,487
York Early College Academy,496,481,473
Young Women's Leadership School,478,465,472


Another way to think about this is that pivoting data in this way requires some sort of data aggregation. For our current endeavor of borough-wise analysis, we need some way to collapse all of the SAT Scores for each section into 5 aggregates, one for each borough. Unfortunately, the `pivot()` method does not support aggregation. 

Thankfully, for aggregate reshaping, Pandas offers an alternative with the `pivot_table()` method which we will see next.

## The `pivot_table()` Method

The `pivot()` method does not support aggregation. If we want to pivot using "Borough" in the index, we need a way to condense all of the data into a Borough-level result. For this we turn to the `pivot_table()` method.
* https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.pivot_table.html

For example, to see the average SAT scores per borough, we can do the following:

In [ ]:
sat.pivot_table(values = 'Score', index = "Borough", columns = "SAT Section")

SAT Section,Math,Reading,Writing
Borough,,,
Bronx,404.357143,402.520408,395.846939
Brooklyn,416.403670,410.761468,403.091743
Manhattan,455.887640,444.932584,439.314607
Queens,462.362319,443.260870,439.855072
Staten Island,486.200000,478.500000,474.300000


This is a very convenient interface - all we're doing is describing what we want the resulting pivot table to look like. 

But wait a minute, none of these values we're seeing here were in the original dataframe. Somehow, all of those values pertaining to the many schools in each individual boroughs got aggregated into a new single value for each SAT Section within each borough. What happened?

By default, the `pivot_table()` function aggregates by calculating the mean, which is specified by the `aggfunc` parameter.

In [ ]:
sat.pivot_table(values = 'Score', index = "Borough", columns = "SAT Section", aggfunc = 'mean')

SAT Section,Math,Reading,Writing
Borough,,,
Bronx,404.357143,402.520408,395.846939
Brooklyn,416.403670,410.761468,403.091743
Manhattan,455.887640,444.932584,439.314607
Queens,462.362319,443.260870,439.855072
Staten Island,486.200000,478.500000,474.300000


Let's look at another example where we want to figure out the average percentage of students tested across boroughs.

In [ ]:
sat.head()

,School Name,Borough,Student Enrollment,Percent Tested,SAT Section,Score
0,A. Philip Randolph Campus High School,Manhattan,1354,74.0,Math,459
1,A. Philip Randolph Campus High School,Manhattan,1354,74.0,Reading,453
2,A. Philip Randolph Campus High School,Manhattan,1354,74.0,Writing,447
3,Abraham Lincoln High School,Brooklyn,2341,54.9,Math,441
4,Abraham Lincoln High School,Brooklyn,2341,54.9,Reading,422


To do this with `pivot_table()`, since there aren't multiple types of "Percent Tested" (as opposed to "SAT Section" which did have multiple types), we don't need to specify columns. We just need to specify the values.

In [ ]:
sat.pivot_table(values = 'Percent Tested', index = "Borough", aggfunc = 'mean')

,Percent Tested
Borough,
Bronx,58.220408
Brooklyn,62.423853
Manhattan,71.197753
Queens,69.036232
Staten Island,67.390000


This question could have also been answered by using a simple `groupby()` analysis.

In [ ]:
sat.groupby('Borough').mean()["Percent Tested"]

Borough
Bronx            58.220408
Brooklyn         62.423853
Manhattan        71.197753
Queens           69.036232
Staten Island    67.390000
Name: Percent Tested, dtype: float64

So, this means that about 58% of students from the Bronx took the SAT, right? Not quite.

A word of caution is that we're taking the average of percentages across multiple schools, completely ignoring the fact that the total student enrollment of each school can be much different from other schools. So be careful what you conclude here. If the average percent tested in the Bronx is 58%, it does not mean that 58% of students in the Bronx took the SAT. 

To illustrate this, consider a hypothetical scenario where one school has 1000 students with 10% (or 100 students) tested, while another school has 5000 students with 60% (or 3000 students) tested. 
* The average percent tested between the two schools is (10% + 60%) / 2 = 35%. 
* However, combined together, the two schools have a total of 6000 students with 3100 students tested, which is 51.6% of students overall. 

Notice how different those two values are, 35% vs. 51.6%  

The takeaway - be very carefuly when making conclusions based on average percentages.

## So What was the Percentage of Students Who Took the SAT Across All Boroughs?

In the previous lecture, we calculated the average percent of SAT-tested students across the boroughs. However, this was an average of percentages - it ignored the actual enrollment numbers at each school. Without that information, we cannot calculate the percent of tested students within each borough.

In this lecture we'll prepare our data to do just that - calculate the percent of students tested across each borough when accounting for the total enrollment.

In [ ]:
sat.head()

,School Name,Borough,Student Enrollment,Percent Tested,SAT Section,Score
0,A. Philip Randolph Campus High School,Manhattan,1354,74.0,Math,459
1,A. Philip Randolph Campus High School,Manhattan,1354,74.0,Reading,453
2,A. Philip Randolph Campus High School,Manhattan,1354,74.0,Writing,447
3,Abraham Lincoln High School,Brooklyn,2341,54.9,Math,441
4,Abraham Lincoln High School,Brooklyn,2341,54.9,Reading,422


Let's start by calculating how many students from each school actually took the exam and adding that as a column to our dataframe. This is a simple multiplication of Student Enrollment and Percent Tested

In [ ]:
sat['SAT Takers'] = (sat['Percent Tested'] * sat['Student Enrollment']) / 100

In [ ]:
sat.head()

,School Name,Borough,Student Enrollment,Percent Tested,SAT Section,Score,SAT Takers
0,A. Philip Randolph Campus High School,Manhattan,1354,74.0,Math,459,1001.960
1,A. Philip Randolph Campus High School,Manhattan,1354,74.0,Reading,453,1001.960
2,A. Philip Randolph Campus High School,Manhattan,1354,74.0,Writing,447,1001.960
3,Abraham Lincoln High School,Brooklyn,2341,54.9,Math,441,1285.209
4,Abraham Lincoln High School,Brooklyn,2341,54.9,Reading,422,1285.209


Great! Now we can simply use `pivot_table()` to determine the average number of SAT takers per borough, right?

In [ ]:
sat.pivot_table(values = 'SAT Takers', index = "Borough", aggfunc = 'mean')

,SAT Takers
Borough,
Bronx,322.523888
Brooklyn,498.704807
Manhattan,477.360876
Queens,747.937551
Staten Island,1271.262500


The above calculation is technically correct, in that Staten Island has the greatest average number of test takers across all of its schools. Does that mean Staten Island had the greatest percentage of test takers? Maybe, but not necessarily. 

To answer that, we need to know that total number of students enrolled within each borough. For each borough, we can divide the sum total of SAT Takers by the sum total of Student Enrollment to get the percentage of test takers.

First let's use `groupby()` to calculate the total student enrollment across each borough.

In [ ]:
sat.groupby('Borough').sum()['Student Enrollment']

Borough
Bronx            159267
Brooklyn         242814
Manhattan        170370
Queens           232437
Staten Island     55380
Name: Student Enrollment, dtype: int64

Let's do the same to calculate the total number of SAT Takers across each borough.

In [ ]:
sat.groupby('Borough').sum()['SAT Takers']

Borough
Bronx             94822.023
Brooklyn         163076.472
Manhattan        127455.354
Queens           154823.073
Staten Island     38137.875
Name: SAT Takers, dtype: float64

Finally, we divide the per-borough SAT Takers by the per-borough Student Enrollment.

In [ ]:
((sat.groupby('Borough').sum()['SAT Takers'] / sat.groupby('Borough').sum()['Student Enrollment']) * 100).nsmallest(10)

Borough
Bronx            59.536516
Queens           66.608618
Brooklyn         67.161066
Staten Island    68.865791
Manhattan        74.810914
dtype: float64

Let's compare this result to the naive average percentage above:

In [ ]:
sat.groupby('Borough').mean()["Percent Tested"].nsmallest(10)

Borough
Bronx            58.220408
Brooklyn         62.423853
Staten Island    67.390000
Queens           69.036232
Manhattan        71.197753
Name: Percent Tested, dtype: float64

These percentages are somewhat close, but not identical. In fact, we see that Queens, not Brooklyn, is has the second-lowest percentage of SAT Takers.

As a final note, a completely algebraically equivalent approach to calculating the percentage of test takers for each boroughs would be to simply divide the average number of test takers at all schools in the borough by the average student enrollment at all schools in the borough.

In [ ]:
sat.groupby('Borough')[['Student Enrollment', 'SAT Takers']].apply(lambda sg: sg['SAT Takers'].mean() * 100 / sg['Student Enrollment'].mean()).nsmallest(10)

Borough
Bronx            59.536516
Queens           66.608618
Brooklyn         67.161066
Staten Island    68.865791
Manhattan        74.810914
dtype: float64

## Replicating Pivot Tables with `groupby()`

The pivot table method is fantastic and has a very intuitive interface - all we need to do is describe the values, index and columns and we get a reshaped dataframe with very little code.

Let's now connect what we've learned here with concepts from previous sections. In this lecture, we'll replicate the pivot table step-by-step using `groupby()`. This gives us an appreciate of how these different methods can be used to achieve similar results.

Let's begin by grouping by Borough and SAT Section:

In [ ]:
sat.groupby(['Borough', 'SAT Section'])

Now we'll apply an aggregation with `agg()`
* an alternative approach would be to use the standard `mean()` function on the entire `groupby` object, select for the `Score` column, then unstack as seen below

In [ ]:
sat.groupby(['Borough', 'SAT Section']).agg({'Score':'mean'})

Score
Borough       SAT Section            
Bronx         Math         404.357143
              Reading      402.520408
              Writing      395.846939
Brooklyn      Math         416.403670
              Reading      410.761468
              Writing      403.091743
Manhattan     Math         455.887640
              Reading      444.932584
              Writing      439.314607
Queens        Math         462.362319
              Reading      443.260870
              Writing      439.855072
Staten Island Math         486.200000
              Reading      478.500000
              Writing      474.300000

Almost there. The only thing we need to do now is pivot the SAT Section column into the column axis, so that we have individual columns for Math, Reading, and Writing. We can accomplish this using `unstack()`

In [ ]:
sat.groupby(['Borough', 'SAT Section']).agg({'Score':'mean'}).unstack('SAT Section')

Score                        
SAT Section          Math     Reading     Writing
Borough                                          
Bronx          404.357143  402.520408  395.846939
Brooklyn       416.403670  410.761468  403.091743
Manhattan      455.887640  444.932584  439.314607
Queens         462.362319  443.260870  439.855072
Staten Island  486.200000  478.500000  474.300000

This is exactly equivalent to the `pivot_table()` method that we used above, reproduced here:

In [ ]:
sat.pivot_table(values = 'Score', index = "Borough", columns = "SAT Section", aggfunc = 'mean')

SAT Section,Math,Reading,Writing
Borough,,,
Bronx,404.357143,402.520408,395.846939
Brooklyn,416.403670,410.761468,403.091743
Manhattan,455.887640,444.932584,439.314607
Queens,462.362319,443.260870,439.855072
Staten Island,486.200000,478.500000,474.300000


The key takeaway here is that all of pivot table functionality can always be replicated by a combination of `groupby()` and aggregate functions. 

## Adding Margins to Pivot Tables for Row- or Column-wise Aggregate Calculations

Let's revisit our borough-wise SAT Score pivot table

In [ ]:
sat.pivot_table(values = 'Score', index = "Borough", columns = "SAT Section", aggfunc = 'mean')

SAT Section,Math,Reading,Writing
Borough,,,
Bronx,404.357143,402.520408,395.846939
Brooklyn,416.403670,410.761468,403.091743
Manhattan,455.887640,444.932584,439.314607
Queens,462.362319,443.260870,439.855072
Staten Island,486.200000,478.500000,474.300000


Sometimes in presentations such as this, it is useful to know the column- or row-wise metrics. For example, what is the average Math score across all NYC high schools? Or, what is the average section score across all schools in Manhatten? 

The Pandas pivot table gives us an easy way to tag such totals into our dataframe. It involves simply setting the `margins` parameter to `True`. This will capture the row-wise and column-wise aggregations for the function you specified in`aggfunc`.

In [ ]:
sat.pivot_table(values = 'Score', index = "Borough", columns = "SAT Section", aggfunc = 'mean', margins = True)

SAT Section,Math,Reading,Writing,All
Borough,,,,
Bronx,404.357143,402.520408,395.846939,400.908163
Brooklyn,416.403670,410.761468,403.091743,410.085627
Manhattan,455.887640,444.932584,439.314607,446.711610
Queens,462.362319,443.260870,439.855072,448.492754
Staten Island,486.200000,478.500000,474.300000,479.666667
All,432.944000,424.504000,418.458667,425.302222


We can verify this by selecting out one borough and testing.

In [ ]:
sat[sat.Borough == 'Manhattan'].Score.mean()

446.7116104868914

Suppose we want to confirm the intersection aggregate score of all boroughs and all SAT sections (in other words, the average SAT score for the entire city), we can simply calculate the mean score.

In [ ]:
sat.Score.mean()

425.3022222222222

Lastly, to bulk-validate the average combined scores for all SAT Sections across all boroughs (the right-most column of the pivot table), we can simply create smaller pivot scores by borough.

In [ ]:
sat.pivot_table(values = 'Score', index = 'Borough')

,Score
Borough,
Bronx,400.908163
Brooklyn,410.085627
Manhattan,446.711610
Queens,448.492754
Staten Island,479.666667


We can also validate the Average score for each SAT section across all boroughs (the bottom row of the pivot table).

In [ ]:
sat.pivot_table(values = 'Score', index = 'SAT Section')

,Score
SAT Section,
Math,432.944000
Reading,424.504000
Writing,418.458667


One thing to note is that the margin columns always apply the same `aggfunc` as the main pivot table. That aggregation is inherited. If we change from "mean" to a different `aggfunc`, the values in the margins will also change accordingly.

In [ ]:
sat.pivot_table(values = 'Score', index = "Borough", columns = "SAT Section", aggfunc = 'max', margins = True)

SAT Section,Math,Reading,Writing,All
Borough,,,,
Bronx,714,672,672,714
Brooklyn,682,608,606,682
Manhattan,754,697,693,754
Queens,701,640,661,701
Staten Island,711,660,670,711
All,754,697,693,754


Lastly, by default, the margins row and column is titled "All" by default. It kind of makes sense because these values reflect aggregates across all members of that axis, in this case across all boroughs or all SAT Sections.

But we do have the ability to change the margin labels by using the `margins_name` parameter.

In [ ]:
sat.pivot_table(values = 'Score', index = "Borough", columns = "SAT Section", aggfunc = 'max', margins = True, margins_name="Grand Total")

SAT Section,Math,Reading,Writing,Grand Total
Borough,,,,
Bronx,714,672,672,714
Brooklyn,682,608,606,682
Manhattan,754,697,693,754
Queens,701,640,661,701
Staten Island,711,660,670,711
Grand Total,754,697,693,754


## MultiIndex Pivot Tables

As we've seen, pivot tables combine key concepts that we are familiar with:
* hierarchically-indexed dataframes
* `groupby()`
* aggregate functions

We can leverage the nice syntax of `pivot_table()` to create multiindex dataframes with relative ease.

Consider our starting our basic pivot table below.

In [ ]:
sat.pivot_table('Score','School Name', 'SAT Section')

SAT Section,Math,Reading,Writing
School Name,,,
A. Philip Randolph Campus High School,459,453,447
ACORN Community High School,379,395,385
Abraham Lincoln High School,441,422,422
Academy for Careers in Television and Film,444,458,444
Academy for College Preparation and Career Exploration (College Board),386,397,393
...,...,...,...
World Journalism Preparatory (College Board),484,491,487
York Early College Academy,496,481,473
Young Women's Leadership School,478,465,472


Suppose we want to squeeze Borough into our index and create a two-level index. The only thing we need to do is to convert the index argument from a single string into a list of strings. Easy!

In [ ]:
sat.pivot_table('Score',['School Name', 'Borough'], 'SAT Section')

,SAT Section,Math,Reading,Writing
School Name,Borough,,,
A. Philip Randolph Campus High School,Manhattan,459,453,447
ACORN Community High School,Brooklyn,379,395,385
Abraham Lincoln High School,Brooklyn,441,422,422
Academy for Careers in Television and Film,Queens,444,458,444
Academy for College Preparation and Career Exploration (College Board),Brooklyn,386,397,393
...,...,...,...,...
World Journalism Preparatory (College Board),Queens,484,491,487
York Early College Academy,Queens,496,481,473
Young Women's Leadership School,Manhattan,478,465,472


Hierarchical indexing isn't restricted to the index. It can also be applied to the columns. All we need to do is swap the arguments between the `index` and `column` parameters.

In [ ]:
sat.pivot_table(values = 'Score', columns = ['School Name', 'Borough'], index = 'SAT Section')

School Name,A. Philip Randolph Campus High School,ACORN Community High School,Abraham Lincoln High School,Academy for Careers in Television and Film,Academy for College Preparation and Career Exploration (College Board),Academy for Conservation and the Environment,Academy for Environmental Leadership,Academy for Health Careers,Academy for Language and Technology,Academy for Scholarship and Entrepreneurship (College Board),Academy for Social Action (College Board),Academy for Young Writers,Academy of American Studies,Academy of Finance and Enterprise,Academy of Hospitality and Tourism,Academy of Innovative Technology,Academy of Medical Technology (College Board),Academy of Urban Planning,Alfred E. Smith Career and Technical Education High School,All City Leadership Secondary School,Antonia Pantoja Preparatory Academy (College Board),"Archimedes Academy for Math, Science, and Technology Applications",Art and Design High School,Arts and Media Preparatory Academy,Astor Collegiate Academy,August Martin High School,Automotive High School,Aviation Career and Technical Education High School,Baccalaureate School for Global Education,Banana Kelly High School,Bard High School Early College,Bard High School Early College Queens,Baruch College Campus High School,Bayside High School,Beacon High School,Bedford Academy High School,Belmont Preparatory High School,Benjamin Banneker Academy,Benjamin N. Cardozo High School,Boys and Girls High School,...,Transit Tech Career and Technical Education High School,Unity Center for Urban Technologies,University Heights Secondary School,University Neighborhood High School,Urban Action Academy,Urban Assembly Academy of Government and Law,Urban Assembly Bronx Academy of Letters,Urban Assembly Gateway School for Technology,Urban Assembly High School of Music and Art,Urban Assembly Institute of Math and Science for Young Women,Urban Assembly New York Harbor School,Urban Assembly School for Applied Math and Science,Urban Assembly School for Careers in Sports,Urban Assembly School for Criminal Justice,Urban Assembly School for Green Careers,Urban Assembly School for Law and Justice,Urban Assembly School for Media Studies,Urban Assembly School for Wildlife Conservation,Urban Assembly School for the Performing Arts,Urban Assembly School of Business for Young Women,Urban Assembly School of Design and Construction,Validus Preparatory Academy: An Expeditionary Learning School,Vanguard High School,Victory Collegiate High School,W. H. Maxwell Career and Technical Education High School,Wadleigh Secondary School for the Performing and Visual Arts,Washington Heights Expeditionary Learning School,West Bronx Academy for the Future,William Cullen Bryant High School,William E. Grady Career and Technical Education High School,Williamsburg High School for Architecture and Design,Williamsburg Preparatory School,Wings Academy,Women's Academy of Excellence,World Academy for Total Community Health High School,World Journalism Preparatory (College Board),York Early College Academy,Young Women's Leadership School,Young Women's Leadership School in Astoria,Young Women's Leadership School in Queens
Borough,Manhattan,Brooklyn,Brooklyn,Queens,Brooklyn,Brooklyn,Brooklyn,Brooklyn,Bronx,Bronx,Manhattan,Brooklyn,Queens,Queens,Brooklyn,Brooklyn,Queens,Brooklyn,Bronx,Brooklyn,Bronx,Bronx,Manhattan,Brooklyn,Bronx,Queens,Brooklyn,Queens,Queens,Bronx,Manhattan,Queens,Manhattan,Queens,Manhattan,Brooklyn,Bronx,Brooklyn,Queens,Brooklyn,...,Brooklyn,Manhattan,Bronx,Manhattan,Brooklyn,Manhattan,Bronx,Manhattan,Brooklyn,Brooklyn,Manhattan,Bronx,Bronx,Brooklyn,Manhattan,Brooklyn,Manhattan,Bronx,Manhattan,Manhattan,Manhattan,Bronx,Manhattan,Brooklyn,Brooklyn,Manhattan,Manhattan,Bronx,Queens,Brooklyn,Brooklyn,Brooklyn,Bronx,Bronx,Brooklyn,Queens,Queens,Manhattan,Queens,Queens
SAT Section,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Math,459,379,441,444,386,381,365,406,371,392,357,382,495,489,374,395,422,392,390,467,43

This gives us a multiindex column axis.

A perhaps easier way to start with a hierarchical index and flip it around to a column multiindex is to transpose the starting dataframe.

In [ ]:
sat.pivot_table(values = 'Score', columns = ['School Name', 'Borough'], index = 'SAT Section').T

,SAT Section,Math,Reading,Writing
School Name,Borough,,,
A. Philip Randolph Campus High School,Manhattan,459,453,447
ACORN Community High School,Brooklyn,379,395,385
Abraham Lincoln High School,Brooklyn,441,422,422
Academy for Careers in Television and Film,Queens,444,458,444
Academy for College Preparation and Career Exploration (College Board),Brooklyn,386,397,393
...,...,...,...,...
World Journalism Preparatory (College Board),Queens,484,491,487
York Early College Academy,Queens,496,481,473
Young Women's Leadership School,Manhattan,478,465,472


And we can transpose it back.

In [ ]:
sat.pivot_table(values = 'Score', columns = ['School Name', 'Borough'], index = 'SAT Section').T.T

School Name,A. Philip Randolph Campus High School,ACORN Community High School,Abraham Lincoln High School,Academy for Careers in Television and Film,Academy for College Preparation and Career Exploration (College Board),Academy for Conservation and the Environment,Academy for Environmental Leadership,Academy for Health Careers,Academy for Language and Technology,Academy for Scholarship and Entrepreneurship (College Board),Academy for Social Action (College Board),Academy for Young Writers,Academy of American Studies,Academy of Finance and Enterprise,Academy of Hospitality and Tourism,Academy of Innovative Technology,Academy of Medical Technology (College Board),Academy of Urban Planning,Alfred E. Smith Career and Technical Education High School,All City Leadership Secondary School,Antonia Pantoja Preparatory Academy (College Board),"Archimedes Academy for Math, Science, and Technology Applications",Art and Design High School,Arts and Media Preparatory Academy,Astor Collegiate Academy,August Martin High School,Automotive High School,Aviation Career and Technical Education High School,Baccalaureate School for Global Education,Banana Kelly High School,Bard High School Early College,Bard High School Early College Queens,Baruch College Campus High School,Bayside High School,Beacon High School,Bedford Academy High School,Belmont Preparatory High School,Benjamin Banneker Academy,Benjamin N. Cardozo High School,Boys and Girls High School,...,Transit Tech Career and Technical Education High School,Unity Center for Urban Technologies,University Heights Secondary School,University Neighborhood High School,Urban Action Academy,Urban Assembly Academy of Government and Law,Urban Assembly Bronx Academy of Letters,Urban Assembly Gateway School for Technology,Urban Assembly High School of Music and Art,Urban Assembly Institute of Math and Science for Young Women,Urban Assembly New York Harbor School,Urban Assembly School for Applied Math and Science,Urban Assembly School for Careers in Sports,Urban Assembly School for Criminal Justice,Urban Assembly School for Green Careers,Urban Assembly School for Law and Justice,Urban Assembly School for Media Studies,Urban Assembly School for Wildlife Conservation,Urban Assembly School for the Performing Arts,Urban Assembly School of Business for Young Women,Urban Assembly School of Design and Construction,Validus Preparatory Academy: An Expeditionary Learning School,Vanguard High School,Victory Collegiate High School,W. H. Maxwell Career and Technical Education High School,Wadleigh Secondary School for the Performing and Visual Arts,Washington Heights Expeditionary Learning School,West Bronx Academy for the Future,William Cullen Bryant High School,William E. Grady Career and Technical Education High School,Williamsburg High School for Architecture and Design,Williamsburg Preparatory School,Wings Academy,Women's Academy of Excellence,World Academy for Total Community Health High School,World Journalism Preparatory (College Board),York Early College Academy,Young Women's Leadership School,Young Women's Leadership School in Astoria,Young Women's Leadership School in Queens
Borough,Manhattan,Brooklyn,Brooklyn,Queens,Brooklyn,Brooklyn,Brooklyn,Brooklyn,Bronx,Bronx,Manhattan,Brooklyn,Queens,Queens,Brooklyn,Brooklyn,Queens,Brooklyn,Bronx,Brooklyn,Bronx,Bronx,Manhattan,Brooklyn,Bronx,Queens,Brooklyn,Queens,Queens,Bronx,Manhattan,Queens,Manhattan,Queens,Manhattan,Brooklyn,Bronx,Brooklyn,Queens,Brooklyn,...,Brooklyn,Manhattan,Bronx,Manhattan,Brooklyn,Manhattan,Bronx,Manhattan,Brooklyn,Brooklyn,Manhattan,Bronx,Bronx,Brooklyn,Manhattan,Brooklyn,Manhattan,Bronx,Manhattan,Manhattan,Manhattan,Bronx,Manhattan,Brooklyn,Brooklyn,Manhattan,Manhattan,Bronx,Queens,Brooklyn,Brooklyn,Brooklyn,Bronx,Bronx,Brooklyn,Queens,Queens,Manhattan,Queens,Queens
SAT Section,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Math,459,379,441,444,386,381,365,406,371,392,357,382,495,489,374,395,422,392,390,467,43

The big takeaway here is that using pivot table syntax, we can create multiindex dataframes along both axes simply by passing in a list of labels instead of single strings.

## Applying Multiple Functions

Recall the pivot where we calculated the average SAT section scores by borough. Since didn't specify an aggregation function, Pandas used "mean" by default. We saw that we can change this by using the `aggfunc` parameter.

In [ ]:
sat.pivot_table(values='Score',index = 'Borough', columns='SAT Section', aggfunc='max')

SAT Section,Math,Reading,Writing
Borough,,,
Bronx,714,672,672
Brooklyn,682,608,606
Manhattan,754,697,693
Queens,701,640,661
Staten Island,711,660,670


But what if we want to apply multiple functions within one single pivot table? For instance if we wanted to see the minimum and maximum values at the same time. Can we do that? Yes we can!

Problem: Show the minimum AND maximum SAT section scores by borough in one pivot table.

We can achieve this very easily be passing a list of function names to the `aggfunc` parameter.

In [ ]:
sat.pivot_table(values='Score',index = 'Borough', columns='SAT Section', aggfunc=['min', 'max'])

min                  max                
SAT Section   Math Reading Writing Math Reading Writing
Borough                                                
Bronx          317     311     292  714     672     672
Brooklyn       319     302     284  682     608     606
Manhattan      344     330     316  754     697     693
Queens         340     320     318  701     640     661
Staten Island  420     427     409  711     660     670

Very nice. We could also use the numpy functions, and also rename the functions!

In [ ]:
sat.pivot_table(values='Score',index = 'Borough', columns='SAT Section', aggfunc=[np.min, np.max]).rename(axis = 1, level = 0, mapper={'amin':'minimum', 'amax':'maximum'})

minimum                 maximum                
SAT Section      Math Reading Writing    Math Reading Writing
Borough                                                      
Bronx             317     311     292     714     672     672
Brooklyn          319     302     284     682     608     606
Manhattan         344     330     316     754     697     693
Queens            340     320     318     701     640     661
Staten Island     420     427     409     711     660     670

## Skill Challenge

#### 1. Starting with the `sat` dataframe, create a pivot table that summarizes the total student enrollment (Student Enrollment) across all 5 boroughs (Borough). Which boroughs have the highest and lowest high school bodies?

This can be achieved using a simply pivot table with Borough as the index and Student Enrollment as values. We'll want to use the "sum" aggregation function to find the total student enrollment.

In [ ]:
sat.pivot_table(index = 'Borough', values = "Student Enrollment", aggfunc='sum').sort_values(by="Student Enrollment", ascending = False)

,Student Enrollment
Borough,
Brooklyn,242814
Queens,232437
Manhattan,170370
Bronx,159267
Staten Island,55380


Brooklyn has the highest student body enrollment, while Statent Island has the lowest. This perhaps makes sense, as Staten Island is the least populated of the NYC boroughs.

#### 2. Modify the pivot table above to also reflect the average student body size (Student Enrollment) across boroughs, within the same pivot table.

This is a simple modification where we just need to add another aggregation function.

In [ ]:
sat.pivot_table(index = 'Borough', values = "Student Enrollment", aggfunc=['sum','mean'])

,sum,mean
,Student Enrollment,Student Enrollment
Borough,,
Bronx,159267,541.724490
Brooklyn,242814,742.550459
Manhattan,170370,638.089888
Queens,232437,1122.884058
Staten Island,55380,1846.000000


If we want to be overachievers we can sort this one as well. Since it's a multiindex column axis, we'll need to pass in a tuple to identify which column to sort by.

In [ ]:
sat.pivot_table(index = 'Borough', values = "Student Enrollment", aggfunc=['sum','mean']).sort_values(by = ("sum",'Student Enrollment'), ascending = False)

,sum,mean
,Student Enrollment,Student Enrollment
Borough,,
Brooklyn,242814,742.550459
Queens,232437,1122.884058
Manhattan,170370,638.089888
Bronx,159267,541.724490
Staten Island,55380,1846.000000


#### 3. Create a pivot table of high schools from the Queens borough (Borough), containing SAT sections scores (SAT Section, Score) as columns and the school name (School Name) as index. Sort the table in descending order by math section scores.

Let's start by creating the pivot table as asked. First, we'll want to subset the dataframe by schools located in Queens.

In [ ]:
sat.loc[sat['Borough'] == 'Queens']

,School Name,Borough,Student Enrollment,Percent Tested,SAT Section,Score,SAT Takers
6,Academy for Careers in Television and Film,Queens,515,95.0,Math,444,489.250
7,Academy for Careers in Television and Film,Queens,515,95.0,Reading,458,489.250
8,Academy for Careers in Television and Film,Queens,515,95.0,Writing,444,489.250
33,Academy of American Studies,Queens,874,84.8,Math,495,741.152
34,Academy of American Studies,Queens,874,84.8,Reading,482,741.152
...,...,...,...,...,...,...,...
1120,Young Women's Leadership School in Astoria,Queens,572,100.0,Reading,464,572.000
1121,Young Women's Leadership School in Astoria,Queens,572,100.0,Writing,477,572.000
1122,Young Women's Leadership School in Queens,Queens,557,90.4,Math,415,503.528
1123,Young Women's Leadership School in Queens,Queens,557,90.4,Reading,420,503.528


Next, we use this subset to create a pivot table as directed.

In [ ]:
sat.loc[sat['Borough'] == 'Queens'].pivot_table(index = 'School Name', columns = "SAT Section", values = "Score")

SAT Section,Math,Reading,Writing
School Name,,,
Academy for Careers in Television and Film,444,458,444
Academy of American Studies,495,482,479
Academy of Finance and Enterprise,489,456,459
Academy of Medical Technology (College Board),422,424,415
August Martin High School,366,372,364
...,...,...,...
William Cullen Bryant High School,466,424,426
World Journalism Preparatory (College Board),484,491,487
York Early College Academy,496,481,473


From here, we simply sort by the Math scores.

In [ ]:
sat.loc[sat['Borough'] == 'Queens'].pivot_table(index = 'School Name', columns = "SAT Section", values = "Score").sort_values(by = 'Math', ascending = False)

SAT Section,Math,Reading,Writing
School Name,,,
Queens High School for the Sciences at York College,701,621,625
Townsend Harris High School,680,640,661
Baccalaureate School for Global Education,633,620,628
Bard High School Early College Queens,631,598,610
Scholars' Academy,588,560,568
...,...,...,...
Middle College High School at LaGuardia Community College,377,389,377
"Queens High School for Information, Research, and Technology",372,362,352
August Martin High School,366,372,364
